# ACL Overview
In the Windows operating system, everything is an object - that is, everything is represented by data structures in memory, that contain members that describe the nature of the data the structure is meant to represent. Obvious things such as files and registry keys are objects, but also more abstract things such as processes and threads are also objects.

Active Directory is no different - AD is primarily a database of components that represent the various identities and relationships in a Windows network. Each object or component in an AD environment is a user, a group, a computer, a printer, a service account, or even a container for any of these.

Again, as in the Windows operating system, Active Directory objects control who has access via Access Control Lists or ACLs (specifically the **D**iscretionary **A**ccess **C**ontrol List or _DACL_), with each Access Control Entry (or ACE) in the ACL specifying an identity (formally called a security principal) and what specific access rights that identity can excise on the object.

Access rights are incredibly powerful - the demonstration below is not exhaustive, but it will be become apparent how destructive a misconfiured ACE on the wrong object can be. Notably however, many of the attacks that are enabled by these access rights are detailed in their own notebooks, making understanding what objects have rights over other objects one of the fundamental tasks in reconaissance, and in understanding the attack paths through an AD environment.  

## WriteProperty
The first right to examine is `WriteProperty`. This right, as the name implies, grants a principal the capability to modify properties on another object.

First, some recon.  We will assume we have access to the account of a user called "Jane Doe" by some initial access technique. This user is part of the following groups:


Next we will construct an LDAP query to look for AD objects that we have this access right on.

_LDAP_MATCHING_RULE_BIT_AND_ = 1.2.840.113556.1.4.803  
_LDAP_MATCHING_RULE_BIT_OR_ = 1.2.840.113556.1.4.804
	
The usage in an LDAP query follows this format `attr:oneoftheaboveOIDs:=value`.

__D:__ indicates a DACL (Discretionary Access Control List)

(A;;WP;;;S-1-5-21-xxxxxxxxx-xxxxxxxxx-xxxxxxxxx-xxxx) is an Access Control Entry (ACE) where:
- __A__ means "Allow"
- __WP__ is the abbreviation for WriteProperty

The last part is the SID (Security Identifier) of the user
https://learn.microsoft.com/en-us/openspecs/windows_protocols/ms-adts/4e638665-f466-4597-93c4-12f2ebfabab5

In [44]:
$domain = New-Object DirectoryServices.DirectoryEntry("LDAP://dc01.aclabuse.lab","aclabuse\jane.doe", "janesPass01")
$searcher = New-Object System.DirectoryServices.DirectorySearcher($domain)
$searcher.Filter = "(&(objectClass=user)(sAMAccountName=jane.doe))"
$searcher.PropertiesToLoad.Add("objectSid")
$sidBytes = $searcher.FindAll().Properties["objectSid"][0]
$userSid = (New-Object System.Security.Principal.SecurityIdentifier($sidBytes, 0)).ToString()
$searcher.Filter = "(&(objectClass=group)(nTSecurityDescriptor:1.2.840.113556.1.4.803:=D:(A;;WP;;;$userSid)))"
$searcher.PropertiesToLoad.Add("name")
$result = $searcher.FindOne().Properties
$result

0
2


## Self

## WriteDACL

## WriteOwner

## ExtendedRights

## GenericWrite

## GenericAll